In [2]:
import pandas as pd
data=pd.read_csv("Customer_Data1.csv", encoding = ('ISO-8859-1'))

In [3]:
data.isna().sum()
df = data[data['basket'].notna()]
order = df.iloc[: , 4:8]
order.iloc[0,:]                
len(order)

107423

In [4]:
list_item=[]
for i in range(len(order)):
    list_item.append(list(order.iloc[i,:]))

In [5]:
#removing nan values
for i in range(len(list_item)):
    list_item[i]=[str(x) for x in list_item[i]]
    list_item[i]=[x for x in list_item[i] if x != 'nan']

In [6]:
list_itemnames=[] 
       
for i in range(len(list_item)):
    s=[]
    for j in range(len(list_item[i])):
        a=list(list_item[i][j].split(","))
        for c in range(len(a)):
            v=list(a[c].partition("-"))
            s.append(v[0])
    list_itemnames.append(s)
            
# here the list is cleaned with na and quantity and can also see how many
#items purchased per transactions

In [7]:
listt=[]     # a list which iterates inside the function and gets appended
temp_list=[]         
for i in range(len(list_itemnames)):
    listt=[]
    for j in range(len(list_itemnames[i])):
        v=list_itemnames[i][j].split(" ")
        if v[-1] =='melon':
            s=[b.replace(v[0] , 'watermelon') for b in v]
            s=s[0]
        else:
            s=(" ").join(v)
            
        listt.append(s.lower().lstrip())
    temp_list.append(listt)

In [8]:
listt=[]    # a list which iterates inside the function and gets appended
temp_list2=[]         
for i in range(len(temp_list)):
    listt=[]
    for j in range(len(temp_list[i])):
        v=temp_list[i][j].split(" ")
        if v[-1] =='pomograntes' or v[-1] =='pomogranates' or v[-1] == 'pomegranetes':
            s=[sub.replace(v[-1] ,'pomegranates') for sub in v]
            s=(" ").join(s)
        else:
            s=(" ").join(v)
            
        listt.append(s.lower().lstrip())
    temp_list2.append(listt)

In [9]:
!pip install apyori    



  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=5d2c5f1e4e19c5e25944d786503773161e0ada319947ef9490f804daf81004c9
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [10]:
from apyori import apriori

In [11]:
rules = apriori(temp_list2, min_support = 0.03, min_confidence = 0.4, min_lift = 1.5, min_length = 2, max_length = 2)
results = list(rules)

results

[RelationRecord(items=frozenset({'badmi mango', 'dasheri mango'}), support=0.08046693910987406, ordered_statistics=[OrderedStatistic(items_base=frozenset({'badmi mango'}), items_add=frozenset({'dasheri mango'}), confidence=0.46525647236126816, lift=2.2898948973913913)]),
 RelationRecord(items=frozenset({'black sapota', 'basmati rice'}), support=0.060331586345568455, ordered_statistics=[OrderedStatistic(items_base=frozenset({'black sapota'}), items_add=frozenset({'basmati rice'}), confidence=0.43552180633021975, lift=1.5052623468167432)]),
 RelationRecord(items=frozenset({'basmati rice', 'cow milk'}), support=0.13093099243178835, ordered_statistics=[OrderedStatistic(items_base=frozenset({'basmati rice'}), items_add=frozenset({'cow milk'}), confidence=0.4525272674624369, lift=1.6086513998682073), OrderedStatistic(items_base=frozenset({'cow milk'}), items_add=frozenset({'basmati rice'}), confidence=0.46543565306595186, lift=1.6086513998682073)]),
 RelationRecord(items=frozenset({'carrot',

In [12]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])


In [13]:
resultsinDataFrame

resultsinDataFrame.to_csv("C:\\Users\\admin\\Desktop\\PROJECT\\model2.csv", index = False)

resultsinDataFrame.nlargest(n = 10, columns = 'Lift')

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,badmi mango,dasheri mango,0.080467,0.465256,2.289895
11,dholka pomegranates,dasheri mango,0.066624,0.434047,2.136288
5,capsicum,red chilli,0.166221,0.506453,1.814456
9,custard apple,kashmir apple,0.100509,0.436772,1.745316
4,brocoli,red onion,0.128203,0.486557,1.725567
20,shimla apple,red chilli,0.089683,0.479733,1.718729
25,spring onion,tomato hybrid,0.098592,0.400613,1.651319
14,green chilli,tomato local,0.191318,0.505236,1.637966
22,toned milk,red chilli,0.073634,0.449406,1.610078
2,basmati rice,cow milk,0.130931,0.452527,1.608651


In [14]:
customer=df.iloc[ : , 1]
customer=customer.reset_index()
customer=customer.iloc[:,1]

In [15]:
all_items = [x for i in temp_list2 for x in i]

In [16]:
#list of all the items one by one

customer_list=[]

for i in range(len(temp_list2)):
    for j in range(len(temp_list2[i])):
         customer_list.append(customer[i])

# here we have now all the items and all the customer ids mapped together

In [17]:
final_data = pd.DataFrame(all_items)
final_data['customer']=customer_list
final_data.columns=['items' , 'customer']
# dataframe created with all the items with their customer ids

all_dummy=pd.get_dummies(final_data , columns=['items'])
unique_dummy= pd.get_dummies(final_data, columns=['items']).groupby('customer', as_index=False).max()
#6966 by 116 . Contains unique items and customers 

In [19]:
unique_dummy

,customer,items_24 mantra brown rice,items_24 mantra organic brown rice,items_24 mantra organic sona masuri brown rice,items_24 mantra organic sona masuri polished rice,items_24 mantra organic sona masuri raw rice,items_24 mantra organic sona masuri rice,items_24 mantra sona masuri rice,items_agro fresh economy sona masuri rice,items_badmi mango,items_basmati brown rice,items_basmati rice,items_basmati rice mogra broken tukda,items_basmati rice super premium,items_basmati rice super premium biryani,items_bemisal basmati rice,items_bemisal basmati rice mothers choice,items_black sapota,items_brocoli,items_brown onion,items_brown rice,items_buffalo milk,items_capsicum,items_carrot,items_charminar long grain basmati rice,items_cherrie tomato,items_cow milk,items_custard apple,items_daawat,items_daawat basmati brown rice,items_daawat basmati rice,items_daawat basmati rice rozana,items_daawat basmati rice super,items_daawat brown rice,items_daawat rozana gold basmati rice,items_daawat sona masuri raw rice,items_daawat sona masuri rice,items_daawat special basmati rice,items_daawat special biryani basmati rice,items_daawat super basmati rice,...,items_more basmati rice,items_more raw sona masuri supreme rice,items_muskat pomegranates,items_natureland organic sona masuri rice,items_nendran banana,items_organic basmati rice,items_organic basmati rice white,items_organic brown sona masuri rice,items_organic sona masuri rice,items_organic tattva,items_organic tattva brown basmati rice,items_organic tattva brown rice,items_poovan banana,items_popular essentials basmati rice brown,items_popular essentials raw basmati rice,items_popular essentials sona masuri raw rice,items_premium basmati rice,items_premium basmati rice brown,items_premium brown rice,items_rasthali banana,items_red bell pepper,items_red chilli,items_red onion,items_ruby pomegranates,items_safe harvest sona masuri rice,items_safe harvest unpolished brown sona masuri rice,items_shimla apple,items_sona masuri boiled rice,items_sona masuri new rice,items_sona masuri rice,items_soursop custard apple,items_special basmati rice,items_spring onion,items_sweet soursop custard apple,items_tomato hybrid,items_tomato local,items_toned milk,items_vmd sona masuri rice supreme,items_watermelon,items_yellow bell pepper
0,CUST1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
1,CUST10,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0
2,CUST100,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1
3,CUST100A,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1
4,CUST100B,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,CUST9M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
6962,CUST9N,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
6963,CUST9O,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0

In [18]:
from sklearn.metrics import pairwise_distances

cosine_sim = 1-pairwise_distances(unique_dummy.iloc[:,1:], metric="cosine")

In [20]:
custindex=pd.Series(unique_dummy.index , index=unique_dummy['customer'])

cust_id=custindex['CUST1']
cust_id

0

In [21]:
def recommend(cust , topn):
    cust_id=custindex[cust]
    cosscore=list(enumerate(cosine_sim[cust_id]))
    cosscore=sorted(cosscore , key=lambda x:x[1] , reverse=True)
    cosscore_new=cosscore[0:topn+1]
    
    cust_idx=[i[0] for i in cosscore_new]
    cust_score=[i[1] for i in cosscore_new]
    
    similar_cust=pd.DataFrame(columns=['Name' , 'Scores'])
    similar_cust['Name']=unique_dummy.loc[cust_idx , 'customer']
    similar_cust['Scores']=cust_score
    print(similar_cust)
    

recommend('CUST1' , topn=10)

        Name    Scores
0      CUST1  1.000000
1988  CUST1H  0.960769
1995  CUST1O  0.960769
1996  CUST1P  0.960769
1985  CUST1E  0.919866
1986  CUST1F  0.919866
1987  CUST1G  0.919866
1989  CUST1I  0.919866
1990  CUST1J  0.919866
1991  CUST1K  0.919866
1993  CUST1M  0.919866


In [22]:
item_dummy=unique_dummy.T
item_dummy=item_dummy.reset_index()
item_dummy.columns=item_dummy.iloc[0,:]
item_dummy=item_dummy.iloc[1:,:].reset_index()
item_dummy=item_dummy.iloc[:,1:]
item_dummy=item_dummy.rename(columns = {'customer':'items'})

item_cosine =1-pairwise_distances(item_dummy.iloc[:,1:] , metric='cosine')
# similarity measure for items 

itemindex=pd.Series(item_dummy.index , index=item_dummy['items'])
itemindex

items
items_24 mantra brown rice                             0
items_24 mantra organic brown rice                     1
items_24 mantra organic sona masuri brown rice         2
items_24 mantra organic sona masuri polished rice      3
items_24 mantra organic sona masuri raw rice           4
                                                    ... 
items_tomato local                                   110
items_toned milk                                     111
items_vmd sona masuri rice supreme                   112
items_watermelon                                     113
items_yellow bell pepper                             114
Length: 115, dtype: int64

In [23]:
def item_recommend(item , topn):
    item_id=itemindex[item]
    cosscore=list(enumerate(item_cosine[item_id]))
    cosscore=sorted(cosscore , key=lambda x:x[1] , reverse=True)
    cosscore_new=cosscore[0:topn+1]
    
    item_idx=[i[0] for i in cosscore_new]
    item_score=[i[1] for i in cosscore_new]
    
    similar_item=pd.DataFrame(columns=['Name' , 'Scores'])
    similar_item['Name']=item_dummy.loc[item_idx , 'items']
    similar_item['Scores']=item_score
    print(similar_item)
    
    
item_recommend('items_organic sona masuri rice' , 5)
item_recommend(98, 5)

                              Name    Scores
83  items_organic sona masuri rice  1.000000
75         items_more basmati rice  0.685994
57   items_india gate basmati rice  0.563602
52                items_gala apple  0.110337
74             items_mammey sapota  0.108078
94           items_rasthali banana  0.104063
                        Name    Scores
98   items_ruby pomegranates  1.000000
113         items_watermelon  0.484706
53        items_green chilli  0.473712
96          items_red chilli  0.460119
22              items_carrot  0.457957
104   items_sona masuri rice  0.457746


In [24]:
!pip install lightfm

     |████████████████████████████████| 310 kB 5.0 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705390 sha256=df443b71022c388ae3bcbb5ace7d84db822ce38e78443f0defe32061b7e0c75c
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [25]:
from lightfm import LightFM

In [26]:
unique_dum=unique_dummy.set_index('customer')

In [27]:
cust_id=list(unique_dum.index)
cust_dict={}
counter=0
for i in cust_id:
  cust_dict[i]=counter
  counter += 1

item_list=list(unique_dum.columns)
item_dict={}
counter=0
for i in item_list:
  item_dict[counter]=i
  counter += 1

In [31]:
import pickle
pickle.dump(cust_dict,open('cust_dict.pkl','wb'))

In [29]:
cust_dict

{'CUST1': 0,
 'CUST10': 1,
 'CUST100': 2,
 'CUST100A': 3,
 'CUST100B': 4,
 'CUST100C': 5,
 'CUST100D': 6,
 'CUST100E': 7,
 'CUST100F': 8,
 'CUST100G': 9,
 'CUST100H': 10,
 'CUST100I': 11,
 'CUST100J': 12,
 'CUST100K': 13,
 'CUST100L': 14,
 'CUST100M': 15,
 'CUST100N': 16,
 'CUST100O': 17,
 'CUST100P': 18,
 'CUST100Q': 19,
 'CUST101': 20,
 'CUST101A': 21,
 'CUST101B': 22,
 'CUST101C': 23,
 'CUST101D': 24,
 'CUST101E': 25,
 'CUST101F': 26,
 'CUST101G': 27,
 'CUST101H': 28,
 'CUST101I': 29,
 'CUST101J': 30,
 'CUST101K': 31,
 'CUST101L': 32,
 'CUST101M': 33,
 'CUST101N': 34,
 'CUST101O': 35,
 'CUST101P': 36,
 'CUST101Q': 37,
 'CUST102': 38,
 'CUST102A': 39,
 'CUST102B': 40,
 'CUST102C': 41,
 'CUST102D': 42,
 'CUST102E': 43,
 'CUST102F': 44,
 'CUST102G': 45,
 'CUST102H': 46,
 'CUST102I': 47,
 'CUST102J': 48,
 'CUST102K': 49,
 'CUST102L': 50,
 'CUST102M': 51,
 'CUST102N': 52,
 'CUST102O': 53,
 'CUST102P': 54,
 'CUST102Q': 55,
 'CUST103': 56,
 'CUST103A': 57,
 'CUST103B': 58,
 'CUST103C': 59,

In [32]:
a=pickle.load(open('/content/cust_dict.pkl','rb'))

In [33]:
a

{'CUST1': 0,
 'CUST10': 1,
 'CUST100': 2,
 'CUST100A': 3,
 'CUST100B': 4,
 'CUST100C': 5,
 'CUST100D': 6,
 'CUST100E': 7,
 'CUST100F': 8,
 'CUST100G': 9,
 'CUST100H': 10,
 'CUST100I': 11,
 'CUST100J': 12,
 'CUST100K': 13,
 'CUST100L': 14,
 'CUST100M': 15,
 'CUST100N': 16,
 'CUST100O': 17,
 'CUST100P': 18,
 'CUST100Q': 19,
 'CUST101': 20,
 'CUST101A': 21,
 'CUST101B': 22,
 'CUST101C': 23,
 'CUST101D': 24,
 'CUST101E': 25,
 'CUST101F': 26,
 'CUST101G': 27,
 'CUST101H': 28,
 'CUST101I': 29,
 'CUST101J': 30,
 'CUST101K': 31,
 'CUST101L': 32,
 'CUST101M': 33,
 'CUST101N': 34,
 'CUST101O': 35,
 'CUST101P': 36,
 'CUST101Q': 37,
 'CUST102': 38,
 'CUST102A': 39,
 'CUST102B': 40,
 'CUST102C': 41,
 'CUST102D': 42,
 'CUST102E': 43,
 'CUST102F': 44,
 'CUST102G': 45,
 'CUST102H': 46,
 'CUST102I': 47,
 'CUST102J': 48,
 'CUST102K': 49,
 'CUST102L': 50,
 'CUST102M': 51,
 'CUST102N': 52,
 'CUST102O': 53,
 'CUST102P': 54,
 'CUST102Q': 55,
 'CUST103': 56,
 'CUST103A': 57,
 'CUST103B': 58,
 'CUST103C': 59,

In [34]:
b=[value for (key,value) in a.items() if key=='CUST148D'][0]

In [35]:
from scipy.sparse import csr_matrix

item_matrix=csr_matrix(unique_dum.values)


In [38]:
unique_dum.values

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [39]:
pickle.dump(item_matrix,open('item_matrix.pkl','wb'))

In [40]:
model=LightFM(loss='warp',learning_rate=0.8)

In [41]:
model=model.fit(item_matrix,epochs=30, num_threads=2)

In [42]:
import numpy as np

In [43]:
def item_recom(id , threshold = 0 , nrec_items=10):
  n_cust,n_items=unique_dum.shape
  user_a=cust_dict[id]
  scores=pd.Series(model.predict(user_a , np.arange(n_items)))
  scores.index=unique_dum.columns
  scores=list(pd.Series(scores.sort_values(ascending=False).index))
  known_items = list(pd.Series(unique_dum.loc[id,:][unique_dum.loc[id,:] > threshold].index).sort_values(ascending=False))
  scores = [x for x in scores if x not in known_items]
  return_score_list = scores[0:nrec_items]
  known_items = list(pd.Series(known_items))
  scores = list(pd.Series(return_score_list))
  print("Items already liked by {}:".format(id))
  counter = 1
  for i in known_items:
    print(str(counter) + '- ' + i)
    counter+=1
  print("\n Recommended Items for {}:".format(id))
  return return_score_list

In [44]:
item_recom('CUST1')

Items already liked by CUST1:
1- items_watermelon
2- items_tomato hybrid
3- items_sona masuri rice
4- items_ruby pomegranates
5- items_red onion
6- items_popular essentials sona masuri raw rice
7- items_kashmir apple
8- items_green chilli
9- items_custard apple
10- items_cow milk
11- items_carrot
12- items_brocoli
13- items_24 mantra organic sona masuri raw rice

 Recommended Items for CUST1:


['items_basmati rice super premium',
 'items_24 mantra brown rice',
 'items_fortune hamesha basmati rice',
 'items_gaajar',
 'items_popular essentials raw basmati rice',
 'items_more basmati rice',
 'items_kohinoor basmati rice dubar',
 'items_basmati brown rice',
 'items_fortune basmati rice super',
 'items_daawat brown rice']

In [45]:
# in this function items id is given as input and users are being 
#recommended which are most likely to purchase that product
def user_recom(id, num_users=10):
  n_cust,n_items=unique_dum.shape
  id_items=np.array(list(item_dict.keys()))
  scores=pd.Series(model.predict( np.arange(n_cust) , np.repeat(id_items.searchsorted(id),n_cust)))
  users=list(unique_dum.index[scores.sort_values(ascending=False).head(num_users).index])
  print('Item id {} is {} and users recommended are as follows:'.format(id , item_dict[id]))
  return users

In [46]:
user_recom(34)

Item id 34 is items_daawat sona masuri raw rice and users recommended are as follows:


['CUST180I',
 'CUST131I',
 'CUST33M',
 'CUST60',
 'CUST135K',
 'CUST15Q',
 'CUST12Q',
 'CUST104',
 'CUST66L',
 'CUST195N']

In [47]:
matrix_size=item_matrix.shape[0]*item_matrix.shape[1]
num_purchases=len(item_matrix.nonzero()[0])
sparsity=100*(1-(num_purchases/matrix_size))
sparsity

86.56018674555918

In [48]:
import random

In [49]:
def make_train(ratings , pct_test=0.2):
  test_set=ratings.copy()
  test_set[test_set !=0]=1

  train_set=ratings.copy()

  nonzero_indices=train_set.nonzero()
  nonzero_pairs=list(zip(nonzero_indices[0],nonzero_indices[1]))

  random.seed(0)

  num_samples=int(np.ceil(pct_test*len(nonzero_pairs)))
  samples=random.sample(nonzero_pairs , num_samples)

  user_inds=[index[0] for index in samples]
  item_inds=[index[1] for index in samples]

  train_set[user_inds,item_inds]
  train_set.eliminate_zeros()

  return train_set,test_set,list(set(user_inds))

In [51]:
product_train, product_test, product_users_altered = make_train(item_matrix, pct_test = 0.2)

In [52]:
product_users_altered

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 18

In [53]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406598 sha256=b328aa8abedd1f24e44bd9d558677daf06c6559d4818369bebb7fe5f4cbaebd3
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [54]:
import implicit

alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((item_matrix*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead
GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


In [ ]:
import pickle
pickle.dump(user_vecs,open('user_vecs.pkl','wb'))
pickle.dump(user_vecs,open('item_vecs.pkl','wb'))

In [56]:
import implicit

alpha = 15
model = implicit.als.AlternatingLeastSquares(factors=20, regularization = 0.1, iterations = 50)
data_con=(item_matrix * alpha).astype('double')
model.fit(data_con)

In [57]:
cust_dict

{'CUST1': 0,
 'CUST10': 1,
 'CUST100': 2,
 'CUST100A': 3,
 'CUST100B': 4,
 'CUST100C': 5,
 'CUST100D': 6,
 'CUST100E': 7,
 'CUST100F': 8,
 'CUST100G': 9,
 'CUST100H': 10,
 'CUST100I': 11,
 'CUST100J': 12,
 'CUST100K': 13,
 'CUST100L': 14,
 'CUST100M': 15,
 'CUST100N': 16,
 'CUST100O': 17,
 'CUST100P': 18,
 'CUST100Q': 19,
 'CUST101': 20,
 'CUST101A': 21,
 'CUST101B': 22,
 'CUST101C': 23,
 'CUST101D': 24,
 'CUST101E': 25,
 'CUST101F': 26,
 'CUST101G': 27,
 'CUST101H': 28,
 'CUST101I': 29,
 'CUST101J': 30,
 'CUST101K': 31,
 'CUST101L': 32,
 'CUST101M': 33,
 'CUST101N': 34,
 'CUST101O': 35,
 'CUST101P': 36,
 'CUST101Q': 37,
 'CUST102': 38,
 'CUST102A': 39,
 'CUST102B': 40,
 'CUST102C': 41,
 'CUST102D': 42,
 'CUST102E': 43,
 'CUST102F': 44,
 'CUST102G': 45,
 'CUST102H': 46,
 'CUST102I': 47,
 'CUST102J': 48,
 'CUST102K': 49,
 'CUST102L': 50,
 'CUST102M': 51,
 'CUST102N': 52,
 'CUST102O': 53,
 'CUST102P': 54,
 'CUST102Q': 55,
 'CUST103': 56,
 'CUST103A': 57,
 'CUST103B': 58,
 'CUST103C': 59,

In [58]:
item_matrix

<6966x115 sparse matrix of type '<class 'numpy.uint8'>'
	with 107665 stored elements in Compressed Sparse Row format>

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [64]:
cust_dict.items()
def rec(cust_id,n):
    cust_index=[value for (key,value) in cust_dict.items() if key==cust_id][0]
    purchased_index=item_matrix[cust_index,:].nonzero()[1]
    purchased_items = []
    for i in purchased_index:
        purchased_items.append(item_dict[i])
    
    pref_vec = item_matrix[cust_index,:].toarray()
    pref_vec = (pref_vec.reshape(-1)) + 1
    pref_vec[pref_vec > 1] = 0
    rec_vector = user_vecs[cust_index,:].dot(item_vecs.T)
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = (pref_vec)*(rec_vector_scaled)
    product_idx = np.argsort(recommend_vector)[::-1][:n]
    rec_list = []
    for i in product_idx:
        rec_list.append(item_dict[i])
    purchased_items = pd.Series(purchased_items)
    return purchased_items, rec_list

In [67]:
z = rec('CUST1',5)

In [68]:
z[0] #items already bought by the customer

0      items_24 mantra organic sona masuri raw rice
1                                     items_brocoli
2                                      items_carrot
3                                    items_cow milk
4                               items_custard apple
5                                items_green chilli
6                               items_kashmir apple
7     items_popular essentials sona masuri raw rice
8                                   items_red onion
9                           items_ruby pomegranates
10                           items_sona masuri rice
11                              items_tomato hybrid
12                                 items_watermelon
dtype: object

In [69]:
z[1] # items that are most likely to buy by the customer

['items_green sapota',
 'items_kesar mango',
 'items_spring onion',
 'items_daawat sona masuri raw rice',
 'items_rasthali banana']